In [1]:
import requests
import copy
import csv
import time
import json
import uuid
import pprint
import asyncio
from typing import List
from urllib import parse
from urllib.parse import quote
import inspect
import datetime

In [2]:
class magic_fstring_function:
    def __init__(self, payload):
        self.payload = payload
        self.cached = None
    def __str__(self):
        if self.cached is None:
            vars = inspect.currentframe().f_back.f_globals.copy()
            vars.update(inspect.currentframe().f_back.f_locals)
            self.cached = self.payload.format(**vars)
        return self.cached

In [3]:
uuid = str(uuid.uuid4())
item_list_api = "https://apis.zigbang.com/v2/items"
describe_list_api = item_list_api + '/list'
item_describe_api = "https://apis.zigbang.com/v3/items?item_ids={item_id}&detail=true"
item_view_url = "https://zigbang.com/home/oneroom/items/{item_id}"

referer = "https://zigbang.com/home/oneroom/subways/414/items"
        
headers = {
    'Host':'apis.zigbang.com',
    'Connection':'keep-alive',
    'Pragma':'no-cache',
    'Cache-Control':'no-cache',
    'Accept':'application/json, text/plain, */*',
    'Origin':'https://zigbang.com',
    'User-Agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Mobile Safari/537.36',
    'DNT':'1',
    'Sec-Fetch-Site':'same-site',
    'Sec-Fetch-Mode':'cors',
    'Referer':'https://zigbang.com/home/oneroom/subways/414/items',
    'Accept-Encoding':'gzip, deflate, br',
    'Accept-Language':'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
}

items = {}

In [4]:
def export_items(items):
    fieldnames = [
        'address1',
        'address2',
        'address3',
        'agent_address1',
        'agent_email',
        'agent_lat',
        'agent_lng',
        'agent_mobile',
        'agent_name',
        'agent_phone',
        'building_type',
        'deposit',
        'description',
        'elevator',
        'floor',
        'floor_all',
        'images',
        'local1',
        'local2',
        'local3',
        'manage_cost',
        'manage_cost_inc',
        'movein_date',
        'near_subways',
        'options',
        'original_user_phone',
        'parking',
        'pets_text',
        'random_location',
        'room_direction_text',
        'room_type',
        'sales_type',
        'service_type',
        'size',
        'size_m2',
        'title',
        'updated_at2',
        'user_email',
        'user_intro',
        'user_mobile',
        'user_name',
        'user_phone',
        'url'
    ]
    
    filtered_item = [] 
    maximum_updated = datetime.timedelta(days=7)
    

    for item in items:
        item = item['item']

        item_updated = datetime.datetime.strptime(item['updated_at2'], '%Y-%m-%d')
        if maximum_updated < datetime.datetime.now() - item_updated:
            continue
        item_id = item['id']
        url = magic_fstring_function(item_view_url)
        item['url'] = str(url)
        new_item = {}
        for fieldname in fieldnames:
            new_item[fieldname] = item[fieldname]
        filtered_item.append(new_item)
    return filtered_item

In [5]:
def describe_room_list(items):
    item_ids = list(items.keys())
    max_idx = int(len(item_ids) / 30)
    if len(item_ids) % 30 != 0:
        max_idx += 1
    
    items = []
    headers['Accept'] = 'application/json'
    headers['Referer'] = 'https://zigbang.com/home/oneroom/subways/37/items'
    
    for i in range(max_idx):
        start_idx = i * 30
        end_idx = min((i+1)*30, len(item_ids))
        sub_ids = item_ids[start_idx:end_idx-1]
        item_id = str(sub_ids).replace(' ', '').replace('\'','')
        api = magic_fstring_function(item_describe_api)
        g = api
        resp = requests.get(
            url=str(api),
            headers=headers,
            timeout=5,
        )
        resp.encoding = 'utf-8'
        result = resp.json()
        resp.close()
        items += result['items']
        time.sleep(1)
        
    return items

In [6]:
def get_room_list(
    items,
    domain: str = "zigbang",
    floor_in: str = "ground",
    geohash: str = "wyd",
    rent_gteq: int = 0,
    service_type_eq: str = "원룸",
):
    resp = requests.get(
        url=item_list_api,
        params={
            "domain": domain,
            "floor_in": floor_in,
            "geohash": geohash,
            "rent_gteq": rent_gteq,
            "service_type_eq": service_type_eq,
        },
        headers=headers,
        timeout=5,
    )
    new_items = json.loads(resp.content.decode('utf-8'))
    resp.close()

    for section in new_items['sections']:
        for item in section['item_ids']:
            items[str(item)] = {}

    return items

In [7]:
items = {}

In [8]:
geos_seoul = [
    'wydm'
] 

for geo in geos_seoul:
    items = get_room_list(items, geohash=geo)
    time.sleep(1)

In [9]:
from itertools import islice

sliceItem = dict(islice(items.items(), 3000))

len(sliceItem)

3000

In [10]:
print(sliceItem)

{'28989276': {}, '29052225': {}, '29155132': {}, '29177844': {}, '29149237': {}, '29204892': {}, '29120389': {}, '29231802': {}, '29095530': {}, '29234985': {}, '29004979': {}, '29109365': {}, '29230752': {}, '26081777': {}, '29156706': {}, '29225955': {}, '29115884': {}, '28594699': {}, '29117929': {}, '29213912': {}, '29096057': {}, '29170560': {}, '28900509': {}, '29198031': {}, '29096401': {}, '29225823': {}, '29164679': {}, '28902239': {}, '29236640': {}, '29193031': {}, '29036905': {}, '29218630': {}, '29192252': {}, '29220477': {}, '29193518': {}, '28995360': {}, '29005380': {}, '29218227': {}, '29221480': {}, '29206685': {}, '29237319': {}, '29054046': {}, '29016188': {}, '29112119': {}, '28963713': {}, '29170461': {}, '29072221': {}, '29160387': {}, '29149283': {}, '28901117': {}, '29193106': {}, '29201585': {}, '28964926': {}, '29212660': {}, '29219113': {}, '29204908': {}, '29174220': {}, '29181976': {}, '28735881': {}, '29193432': {}, '29205193': {}, '29184799': {}, '291067

In [11]:
result = describe_room_list(sliceItem)

In [12]:
len(result)

2900

In [13]:
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(result[0]['item'])

{ '_floor': '5층',
  'address1': '경기도 구리시 수택동',
  'address2': '433-44',
  'address3': None,
  'agent_address1': '경기도 구리시 검배로48번길 9',
  'agent_comment': '',
  'agent_email': 'leelucia0556@naver.com',
  'agent_lat': 37.5968597477281,
  'agent_lng': 127.144015269118,
  'agent_local1': '경기도',
  'agent_local2': '구리시',
  'agent_mobile': '010-2260-8844',
  'agent_name': '트윈팰리스공인중개사사무소(이경순)',
  'agent_no': 10738,
  'agent_phone': '031-564-8844',
  'bjd_code': '',
  'bonbun_code': '433',
  'bubun_code': '44',
  'building': None,
  'building_type': '다세대건물',
  'contract': '경기도',
  'deposit': 27500,
  'description': '- 수택동 수택사거리인근에 위치한 신축 빌라입니다.\r\n'
                 '- 방1 거실 구조로 되어있습니다.\r\n'
                 '- 전세자금대출 80% 가능합니다.\r\n'
                 '- 신축 첫입주 매물입니다.\n'
                 '21.10.06에 직방에 등록된 매물입니다.',
  'description_og': '- 수택동 수택사거리인근에 위치한 신축 빌라입니다.\r\n'
                    '- 방1 거실 구조로 되어있습니다.\r\n'
                    '- 전세자금대출 80% 가능합니다.\r\n'
                    '- 신축 첫입주 매물입니다.\n'

In [14]:
filtered_result = export_items(result)

In [15]:
len(filtered_result)


1773

In [16]:
def get_full_address(item):
    addr = item['address1']
    addr += ' '
    addr += item['address2'] or ''
    addr += ' '
    addr += item['address3'] or ''
    return addr

In [17]:
def get_location_lat(item):
    locationList = item['random_location'].split(',')

    lat = locationList[0][0:9]
    
    return lat

def get_location_lng(item):
    locationList = item['random_location'].split(',')

    lng = locationList[1][0:10]
    
    return lng

In [18]:
geo_based_dict = {}

In [19]:
for item in filtered_result:
    full_addr = get_full_address(item)
    item['address'] = full_addr
    if full_addr not in geo_based_dict:
        geo_based_dict[full_addr] = []
    geo_based_dict[full_addr].append(item)
    lat = get_location_lat(item)
    item['lat'] = lat
    lng = get_location_lng(item)
    item['lng'] = lng

In [20]:
first_key = next(iter(geo_based_dict))
pp.pprint(geo_based_dict[first_key][0])

{ 'address': '서울시 관악구 봉천동 1687-16 ',
  'address1': '서울시 관악구 봉천동',
  'address2': '1687-16',
  'address3': None,
  'agent_address1': '서울특별시 관악구 솔밭로2길 21 (봉천동)',
  'agent_email': 'sj911125@gmail.com',
  'agent_lat': 37.4778429960848,
  'agent_lng': 126.963898887231,
  'agent_mobile': '010-5492-4992',
  'agent_name': '주식회사프로부동산중개법인(이상현)',
  'agent_phone': '02-889-1212',
  'building_type': '다세대건물',
  'deposit': 15000,
  'description': '▶[ 매물정보 ]\n'
                 '☞해당호실은 단독주택외1 입니다\n'
                 '☞ 현재 보시는 매물 사진은 제가 직접 현장에서 촬영한 사진입니다\n'
                 '\n'
                 '☞ 방은 전체적으로 깔끔한 인테리어와 화이트톤빌트인이 되어있습니다.\n'
                 '\n'
                 '☞ 현관도어문, 전자도어락 , CCTV, 비디오폰 되어있습니다.\n'
                 '\n'
                 '▶[위치 및 교통]\n'
                 '\n'
                 '☞ 강남권으로 출퇴근 하시는분들과 주변 대학교로 가는 교통편이 많습니다.\n'
                 '\n'
                 '☞ 바로 대로변 인근으로 버스타시기도 용이합니다.\n'
                 '\n'
                 '☞ 서울대,중앙대,숭실대 학생들에게도 좋은위치에 있어 통학이 편리합니다. \n'
 

In [21]:
double_list = []
for addr in geo_based_dict.keys():
    double_list.append(geo_based_dict[addr][0])
double_list.sort(key=lambda x: x['size'], reverse=True)

In [22]:
with open('oneroom.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = [
        'address',
        'address1',
        'address2',
        'address3',
        'agent_address1',
        'agent_email',
        'agent_lat',
        'agent_lng',
        'agent_mobile',
        'agent_name',
        'agent_phone',
        'building_type',
        'deposit',
        'description',
        'elevator',
        'floor',
        'floor_all',
        'images',
        'lat',
        'lng',
        'local1',
        'local2',
        'local3',
        'manage_cost',
        'manage_cost_inc',
        'movein_date',
        'near_subways',
        'options',
        'original_user_phone',
        'parking',
        'pets_text',
        'random_location',
        'room_direction_text',
        'room_type',
        'sales_type',
        'service_type',
        'size',
        'size_m2',
        'title',
        'updated_at2',
        'user_email',
        'user_intro',
        'user_mobile',
        'user_name',
        'user_phone',
        'url'
    ]
    writer = csv.DictWriter(csvfile, extrasaction='ignore', fieldnames=fieldnames)
    writer.writeheader()
    for item in double_list:
        writer.writerow(item)

In [ ]:
import pymysql
import csv

conn = pymysql.connect(host='localhost', user='khweb', password='khWeb@123', db='mybang_db', charset='utf8')
curs = conn.cursor()
sql = "insert into oneroom (id,deposit,agent_name,agent_mobile,floor,address,size,size_m2,status,room_type,manage_cost_inc,url,upddt) values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s, now())"
f = open('crawling_data.csv', 'r', encoding='utf-8')
rd = csv.reader(f)

#title, description은 이모티콘 인식못하는 에러 -> 일단 제외
for line in rd:
    curs.execute(sql, (line[0], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13]))

conn.commit()
conn.close()
f.close()